In [1]:
# download shakespeare dataset
#!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [2]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print("text length", len(text))

text length 1115394


In [4]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [6]:
# tokenize - convert raw text/chars to some sequence of integers accoring to some vocabulary

In [7]:
# create mapping from chars to ints (and vice versa)
stoi = { ch:i for i, ch in enumerate(chars) }
itos = { i:ch for i, ch in enumerate(chars) }

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("hey there"))
print(decode(encode("hey there")))

[46, 43, 63, 1, 58, 46, 43, 56, 43]
hey there


In [8]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:1000])

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
        47, 59, 57,  1, 47, 57,  1, 41, 

In [9]:
# split into train-val
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [10]:
block_size = 8
train_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [11]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {decode(list(context.numpy()))}, the target is {decode(list([target.numpy().item()]))}")

when input is F, the target is i
when input is Fi, the target is r
when input is Fir, the target is s
when input is Firs, the target is t
when input is First, the target is  
when input is First , the target is C
when input is First C, the target is i
when input is First Ci, the target is t


In [12]:
torch.manual_seed(42)
batch_size = 4 # how many independent sequences will we process in parallel -> batch dimension
block_size = 8 # what is the maximum context length for prediction -> time dimension

def get_batch(split):
    if split == 'train':
        data = train_data
    else:
        data = val_data
        
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print(xb)
print(yb)

tensor([[57,  1, 46, 47, 57,  1, 50, 53],
        [ 1, 58, 46, 43, 56, 43,  1, 41],
        [17, 26, 15, 17, 10,  0, 32, 53],
        [57, 58,  6,  1, 61, 47, 58, 46]])
tensor([[ 1, 46, 47, 57,  1, 50, 53, 60],
        [58, 46, 43, 56, 43,  1, 41, 39],
        [26, 15, 17, 10,  0, 32, 53,  1],
        [58,  6,  1, 61, 47, 58, 46,  0]])


In [13]:
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when input is {context.tolist()} the target: {target}")

when input is [57] the target: 1
when input is [57, 1] the target: 46
when input is [57, 1, 46] the target: 47
when input is [57, 1, 46, 47] the target: 57
when input is [57, 1, 46, 47, 57] the target: 1
when input is [57, 1, 46, 47, 57, 1] the target: 50
when input is [57, 1, 46, 47, 57, 1, 50] the target: 53
when input is [57, 1, 46, 47, 57, 1, 50, 53] the target: 60
when input is [1] the target: 58
when input is [1, 58] the target: 46
when input is [1, 58, 46] the target: 43
when input is [1, 58, 46, 43] the target: 56
when input is [1, 58, 46, 43, 56] the target: 43
when input is [1, 58, 46, 43, 56, 43] the target: 1
when input is [1, 58, 46, 43, 56, 43, 1] the target: 41
when input is [1, 58, 46, 43, 56, 43, 1, 41] the target: 39
when input is [17] the target: 26
when input is [17, 26] the target: 15
when input is [17, 26, 15] the target: 17
when input is [17, 26, 15, 17] the target: 10
when input is [17, 26, 15, 17, 10] the target: 0
when input is [17, 26, 15, 17, 10, 0] the targ

In [14]:
print(xb)

tensor([[57,  1, 46, 47, 57,  1, 50, 53],
        [ 1, 58, 46, 43, 56, 43,  1, 41],
        [17, 26, 15, 17, 10,  0, 32, 53],
        [57, 58,  6,  1, 61, 47, 58, 46]])


In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(42)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each otken directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets=None):
        # idx and targets are both (B, T) tensor of integers
        logits = self.token_embedding_table(idx) # (B, T, C)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)

            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(idx)
        
            logits = logits[:, -1, :]
            
            probs = F.softmax(logits, dim=-1)
            
            idx_next = torch.multinomial(probs, num_samples=1)
            
            idx = torch.cat((idx, idx_next), dim=1)
        return idx
            
            
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)

print(logits.shape)
print(loss)

idx = torch.zeros((1, 1,), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8865, grad_fn=<NllLossBackward>)

o$,q&IWqW&xtCjaB?ij&bYRGkF?b; f ,CbwhtERCIfuWr,DzJERjhLlVaF&EjffPHDFcNoGIG'&$qXisWTkJPw
 ,b Xgx?D3sj


In [16]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)


In [17]:
batch_size = 32
for steps in range(10000):
    xb, yb = get_batch('train')
    
    logits, loss = m(xb, yb)
    
    optimizer.zero_grad(set_to_none=True)
    
    loss.backward()
    optimizer.step()
    
    
print(loss.item())

2.5233194828033447


In [18]:
print(decode(m.generate(idx, max_new_tokens=200)[0].tolist()))


QUDUThe chas.
F lisen tabr:
LI mus nk,
A: al l ayo cenghe's therinvar,
TEsen ithawaneit at islinerainy atsomo clour pad d wikn h,
HYy my Tholes:
it GBy ke m vilou xthazinderand llo chee lond Cld this 


## Mathematical trick in self-attention


In [19]:
torch.manual_seed(42)

B, T, C = 4, 8, 2
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

we want $x[b, t] = mean_{i<=t} x[b,i]$

In [24]:
# version 1: for loop
xbow = torch.zeros((B, T, C)) # bag of words
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1]
        xbow[b, t] = torch.mean(xprev, 0)
        
x[0], xbow[0]

(tensor([[ 1.9269,  1.4873],
         [ 0.9007, -2.1055],
         [ 0.6784, -1.2345],
         [-0.0431, -1.6047],
         [-0.7521,  1.6487],
         [-0.3925, -1.4036],
         [-0.7279, -0.5594],
         [-0.7688,  0.7624]]),
 tensor([[ 1.9269,  1.4873],
         [ 1.4138, -0.3091],
         [ 1.1687, -0.6176],
         [ 0.8657, -0.8644],
         [ 0.5422, -0.3617],
         [ 0.3864, -0.5354],
         [ 0.2272, -0.5388],
         [ 0.1027, -0.3762]]))

In [35]:
%%timeit
# version 2: mat mul
weights = torch.tril(torch.ones(T, T))
weights = weights / weights.sum(1, keepdim=True)
xbow2 = weights @ x # (T, T) @ (B, T, C) ---> (B, T, C)

32 µs ± 3.7 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [38]:
%%timeit
# version 3: use softmax
tril = torch.tril(torch.ones(T, T))
weights = torch.zeros((T, T))
weights = weights.masked_fill(tril==0, float('-inf'))
weights = F.softmax(weights, dim=-1)
xbow3 = weights @ x

46.6 µs ± 1.79 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [45]:
# version 4: self attention
torch.manual_seed(42)
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

# single head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(x) # (B, T, head_size)
q = query(x) # (B, T, head_size)

weights = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16  T) -> (B, T, T)
weights = weights * (head_size ** -0.5)
                    
tril = torch.tril(torch.ones(T, T))
weights = weights.masked_fill(tril==0, float('-inf'))
weights = F.softmax(weights, dim=-1)

v = value(x)

out = weights @ v

weights[0], out[0]

(tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4106, 0.5894, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3657, 0.2283, 0.4061, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2168, 0.2759, 0.2204, 0.2870, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2553, 0.1697, 0.1548, 0.2341, 0.1861, 0.0000, 0.0000, 0.0000],
         [0.1318, 0.2060, 0.1405, 0.1917, 0.1949, 0.1351, 0.0000, 0.0000],
         [0.2137, 0.0978, 0.2374, 0.1025, 0.1418, 0.0838, 0.1230, 0.0000],
         [0.0852, 0.1047, 0.0824, 0.1376, 0.1015, 0.1900, 0.1780, 0.1206]],
        grad_fn=<SelectBackward>),
 tensor([[ 0.7630, -0.2412, -0.4150,  0.3833,  0.5740, -1.6738,  0.7954,  0.6872,
          -0.3848,  0.5073, -0.5312, -0.1221,  0.0445,  1.2169,  0.9940,  1.5281],
         [ 0.5029, -0.1326, -0.6701, -0.2706,  0.2883, -0.7521,  0.2712, -0.3253,
          -0.1462,  0.3669,  0.0978,  0.0339,  0.1174,  0.7104,  0.5455,  0.3296],
         [ 0.2792, -0.0053, -0.278

In [44]:
out

tensor([[[ 7.6297e-01, -2.4118e-01, -4.1502e-01,  3.8330e-01,  5.7404e-01,
          -1.6738e+00,  7.9543e-01,  6.8723e-01, -3.8477e-01,  5.0733e-01,
          -5.3124e-01, -1.2214e-01,  4.4479e-02,  1.2169e+00,  9.9396e-01,
           1.5281e+00],
         [ 5.0290e-01, -1.3257e-01, -6.7006e-01, -2.7060e-01,  2.8833e-01,
          -7.5205e-01,  2.7124e-01, -3.2529e-01, -1.4619e-01,  3.6692e-01,
           9.7772e-02,  3.3935e-02,  1.1736e-01,  7.1044e-01,  5.4552e-01,
           3.2963e-01],
         [ 2.7921e-01, -5.2619e-03, -2.7881e-01,  1.7815e-01,  5.4466e-01,
          -8.1475e-01,  2.1705e-01, -8.7892e-02, -3.2445e-01,  3.3359e-01,
          -5.2651e-02, -1.6295e-01, -3.0752e-01,  6.0044e-01,  8.2317e-01,
           7.7139e-01],
         [ 1.0908e-01,  1.7385e-01, -1.8134e-01,  7.3269e-02,  1.5080e-01,
          -5.3500e-01, -3.4226e-02, -4.6532e-01, -2.3805e-01,  2.6292e-01,
           6.1617e-02,  8.5927e-02,  1.1182e-01,  4.9281e-01,  3.9407e-01,
           2.8867e-01],
    